In [1]:
import sys
sys.path.append("/Users/aaron/phd/impact-of-contamination-on-taxonomy")

In [2]:
from workflow.fastani_contig_split.d_report_results import FastAniContigSplitReportResultsFastAni
from collections import Counter

In [3]:
DF_RESULTS = FastAniContigSplitReportResultsFastAni().output().read_cached()


# Subset the results to bacteria only
DF_RESULTS = DF_RESULTS[DF_RESULTS['expected_tax'].apply(lambda x: x.startswith('d__Bacteria'))]

DF_RESULTS.head()

,gid,expected_tax,keep_tax,disc_tax,keep_rank_agree,disc_rank_agree,keep_sp_rep,disc_sp_rep,keep_ani,disc_ani,keep_af,disc_af,keep_type,disc_type,keep_same_as_207,disc_same_as_207,keep_same_as_disc
0,GCA_000143435.1,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,s,s,GCF_001435955.1,GCF_001435955.1,97.3962,97.3042,0.8174,0.7677,sp_rep,sp_rep,True,True,True
1,GCA_000153745.1,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,0.0000,0.0000,0.0000,no_ani,no_ani,True,True,True
2,GCA_000155005.1,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,NaN,s,NaN,GCF_003697165.2,NaN,96.5879,0.0000,0.7680,0.0000,sp_rep,no_af,True,False,False
3,GCA_000161815.1,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...,s,s,GCF_001681715.1,GCF_001681715.1,97.1446,96.9094,0.9309,0.8098,sp_rep,sp_rep,True,True,True
4,GCA_000166735.2,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,0.0000,0.0000,0.0000,no_ani,no_ani,True,True,True


In [ ]:
from workflow.gunc_helper.aggregate_max_css_level_gunc import AggregateMaxCssLevelGtdbR95

AggregateMaxCssLevelGtdbR95()

In [4]:
def get_gids_with_diff_sp_assignments():
    keep_diff, disc_diff = set(), set()
    for row in DF_RESULTS.itertuples():
        if not row.keep_same_as_207:
            keep_diff.add(row.gid)
        if not row.disc_same_as_207:
            disc_diff.add(row.gid)
    return keep_diff, disc_diff

S_GIDS_KEEP_DIFF_ASSIGN, S_GIDS_DISC_DIFF_ASSIGN = get_gids_with_diff_sp_assignments()

In [5]:
from workflow.gunc_helper.aggregate_max_css_level_merged import AggregateMaxCssLevelMerged

DF_CSS = AggregateMaxCssLevelMerged().output().read_cached()
DF_CSS.head()

[11/01/2023 11:36:41] - 1.9MiB = /srv/home/uqamussi/projects/gunc-chimeras/output/gunc_helper/gunc_merged_max_css_level.h5 -> /tmp/gunc-cache/srv/home/uqamussi/projects/gunc-chimeras/output/gunc_helper/gunc_merged_max_css_level.h5


,n_genes_called,n_genes_mapped,n_contigs,taxonomic_level,proportion_genes_retained_in_major_clades,genes_retained_index,clade_separation_score,contamination_portion,n_effective_surplus_clades,mean_hit_identity,reference_representation_score,pass.GUNC,source
gid,,,,,,,,,,,,,
GCA_000143435.1,2189,2110,4,genus,0.89,0.86,0.55,0.03,0.06,0.95,0.82,False,gtdb
GCA_000153745.1,2202,2165,2,family,0.92,0.90,0.49,0.06,0.12,0.67,0.61,False,progenomes
GCA_000155005.1,6879,6313,84,genus,0.87,0.79,0.79,0.10,0.22,0.97,0.77,False,gtdb
GCA_000161815.1,4499,4335,8,genus,0.80,0.77,0.50,0.19,0.50,0.95,0.74,False,progenomes
GCA_000166735.2,3414,3247,793,genus,0.94,0.90,0.57,0.02,0.05,0.93,0.83,False,progenomes


In [6]:
FAIL_GIDS = frozenset(DF_CSS.index)

In [7]:
from workflow.external.gtdb_metadata import GtdbMetadataR207

DF_META = GtdbMetadataR207().output().read_cached()
DF_META.head()

,ambiguous_bases,checkm_completeness,checkm_contamination,checkm_marker_count,checkm_marker_lineage,checkm_marker_set_count,checkm_strain_heterogeneity,coding_bases,coding_density,contig_count,...,gtdb_type_species_of_genus,ncbi_taxonomy,ncbi_taxonomy_unfiltered,domain,phylum,class,order,family,genus,species
gid,,,,,,,,,,,,,,,,,,,,,
GCA_000006155.2,1916,93.12,0.0,1171,g__Bacillus (UID902),324,0.0,4305660,80.178992,426,...,f,d__Bacteria;p__Firmicutes;c__Bacilli;o__Bacill...,d__Bacteria;x__Terrabacteria group;p__Firmicut...,d__Bacteria,p__Firmicutes,c__Bacilli,o__Bacillales,f__Bacillaceae_G,g__Bacillus_A,s__Bacillus_A anthracis
GCA_000007325.1,1,99.95,0.0,149,k__Bacteria (UID2329),89,0.0,1973459,90.754610,1,...,t,d__Bacteria;p__Fusobacteria;c__Fusobacteriia;o...,d__Bacteria;p__Fusobacteria;c__Fusobacteriia;o...,d__Bacteria,p__Fusobacteriota,c__Fusobacteriia,o__Fusobacteriales,f__Fusobacteriaceae,g__Fusobacterium,s__Fusobacterium nucleatum
GCA_000007385.1,0,99.82,0.0,481,c__Gammaproteobacteria (UID4202),276,0.0,4190634,84.805944,1,...,f,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,d__Bacteria,p__Proteobacteria,c__Gammaproteobacteria,o__Xanthomonadales,f__Xanthomonadaceae,g__Xanthomonas,s__Xanthomonas oryzae
GCA_000008085.1,0,73.13,0.0,149,k__Archaea (UID2),107,0.0,462902,94.299480,1,...,t,d__Archaea;p__Nanoarchaeota;c__;o__Nanoarchaea...,d__Archaea;x__DPANN group;p__Nanoarchaeota;o__...,d__Archaea,p__Nanoarchaeota,c__Nanoarchaeia,o__Nanoarchaeales,f__Nanoarchaeaceae,g__Nanoarchaeum,s__Nanoarchaeum equitans
GCA_000008885.1,0,100.00,0.0,134,k__Bacteria (UID2495),80,0.0,617456,87.831079,2,...,f,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,d__Bacteria,p__Proteobacteria,c__Gammaproteobacteria,o__Enterobacterales_A,f__Enterobacteriaceae_A,g__Wigglesworthia,s__Wigglesworthia glossinidia_A


In [8]:
from collections import defaultdict


def get_species_to_gids():
    out = defaultdict(set)
    for row in DF_META.itertuples():
        if row.domain == 'd__Bacteria':
            out[row.species].add(row.Index)
    return out

D_SPECIES_TO_GID = get_species_to_gids()

In [11]:
# Get the number of genomes (pass/fail) in each species
import pandas as pd

def get_n_genomes_in_each_species_pass_fail():
    rows = list()
    for species, gids in D_SPECIES_TO_GID.items():
        rows.append({
            'species': species,
            'n_genomes_in_sp': len(gids),
            'n_failed_genomes_in_sp': len(gids.intersection(FAIL_GIDS)),
            'n_pass_genomes_in_sp': len(gids - FAIL_GIDS),
            'n_failed_genomes_with_diff_sp_assignment_on_keep_half': len(gids.intersection(S_GIDS_KEEP_DIFF_ASSIGN)),
            'n_failed_genomes_with_diff_sp_assignment_on_chimeric_half':  len(gids.intersection(S_GIDS_DISC_DIFF_ASSIGN)),
            'failed_genomes_with_diff_sp_assignment_on_keep_half': ','.join(sorted(gids.intersection(S_GIDS_KEEP_DIFF_ASSIGN))),
            'failed_genomes_with_diff_sp_assignment_on_chimeric_half': ','.join(sorted(gids.intersection(S_GIDS_DISC_DIFF_ASSIGN)))
        })

    return pd.DataFrame(rows)

D_SP_TO_PASS_FAIL = get_n_genomes_in_each_species_pass_fail()
D_SP_TO_PASS_FAIL.head()


,species,n_genomes_in_sp,n_failed_genomes_in_sp,n_pass_genomes_in_sp,n_failed_genomes_with_diff_sp_assignment_on_keep_half,n_failed_genomes_with_diff_sp_assignment_on_chimeric_half,failed_genomes_with_diff_sp_assignment_on_keep_half,failed_genomes_with_diff_sp_assignment_on_chimeric_half
0,s__Bacillus_A anthracis,417,1,416,0,0,,
1,s__Fusobacterium nucleatum,13,0,13,0,0,,
2,s__Xanthomonas oryzae,364,5,359,0,0,,
3,s__Wigglesworthia glossinidia_A,1,0,1,0,0,,
4,s__Phytoplasma sp000009845,11,0,11,0,0,,


In [10]:
D_SP_TO_PASS_FAIL.to_csv('/tmp/species_clustering_results.tsv', sep='\t', index=False)

1091